# Import libraries

In [73]:
import boto3
import sagemaker
import warnings, requests, zipfile, io
import pandas as pd
import os
warnings.simplefilter('ignore')

# Import file from s3

## Import and concatenate dataset from data source 1

In [2]:
# Define S3 bucket and file details
bucket = 'big-data-rmit-asm3'
prefix = 'data/'

In [3]:
file_names = ["humidity", "pressure", "temperature", "weather_description", "wind_direction", "wind_speed"]

In [4]:
us_cities = [
    "San Francisco",  # California
    "Los Angeles",  # California
    "San Diego",  # California
    "San Antonio",  # Texas
    "Dallas",  # Texas
    "Houston",  # Texas
    "Minneapolis",  # Minnesota
    "Chicago",  # Illinois
    "Indianapolis",  # Indiana
    "Charlotte",  # North Carolina
]

In [5]:
weather_df_1 = None

In [6]:
for file_name in file_names:
    file_path = 's3://{}/{}'.format(bucket, prefix) + file_name + '.csv'
    df = pd.read_csv(file_path)
    df = df[['datetime'] + us_cities]
    df = df.melt(id_vars=['datetime'], var_name='cities', value_name= file_name)
    if weather_df_1 is None:
        # For the first dataframe, just assign it to merged_df
        weather_df_1 = df
    else:
        # Merge the current dataframe with the existing merged_df
        weather_df_1 = pd.merge(weather_df_1, df, on=['datetime', 'cities'], how='inner')

In [7]:
weather_df_1.drop({"wind_direction"}, axis=1, inplace=True)

In [8]:
weather_df_1.rename({"datetime": "Weather_Timestamp", "cities": "City", "humidity": "Humidity(%)", "wind_speed": "Wind_Speed(m/s)", "weather_description": "Weather_Condition", "temperature": "Temperature(K)", "pressure": "Pressure(hPa)"},axis=1, inplace =True)

In [9]:
weather_df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 452530 entries, 0 to 452529
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Weather_Timestamp  452530 non-null  object 
 1   City               452530 non-null  object 
 2   Humidity(%)        447185 non-null  float64
 3   Pressure(hPa)      450168 non-null  float64
 4   Temperature(K)     451699 non-null  float64
 5   Weather_Condition  451728 non-null  object 
 6   Wind_Speed(m/s)    451721 non-null  float64
dtypes: float64(4), object(3)
memory usage: 24.2+ MB


In [10]:
weather_df_1.shape

(452530, 7)

In [11]:
weather_df_1.head()

,Weather_Timestamp,City,Humidity(%),Pressure(hPa),Temperature(K),Weather_Condition,Wind_Speed(m/s)
0,2012-10-01 12:00:00,San Francisco,NaN,NaN,NaN,NaN,NaN
1,2012-10-01 13:00:00,San Francisco,88.0,1009.0,289.480000,light rain,2.0
2,2012-10-01 14:00:00,San Francisco,87.0,1009.0,289.474993,sky is clear,2.0
3,2012-10-01 15:00:00,San Francisco,86.0,1009.0,289.460618,sky is clear,2.0
4,2012-10-01 16:00:00,San Francisco,85.0,1009.0,289.446243,sky is clear,2.0


## Import data source 2

In [12]:
US_accidents_data_key = prefix + 'US_Accidents_March23.csv'

In [13]:
US_accidents_data_path = 's3://{}/{}'.format(bucket, US_accidents_data_key)

In [14]:
us_accidents_df = pd.read_csv(US_accidents_data_path)

In [15]:
us_accident_df_copy = us_accidents_df.copy()

In [16]:
us_accident_df_copy.head()

,ID,Source,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,Source2,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,Source2,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,Source2,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,0.01,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,Source2,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,0.01,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,Source2,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,0.01,...,False,False,False,False,True,False,Day,Day,Day,Day


In [17]:
us_accident_df_copy.shape

(7728394, 46)

In [18]:
us_accident_df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7728394 entries, 0 to 7728393
Data columns (total 46 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   Severity               int64  
 3   Start_Time             object 
 4   End_Time               object 
 5   Start_Lat              float64
 6   Start_Lng              float64
 7   End_Lat                float64
 8   End_Lng                float64
 9   Distance(mi)           float64
 10  Description            object 
 11  Street                 object 
 12  City                   object 
 13  County                 object 
 14  State                  object 
 15  Zipcode                object 
 16  Country                object 
 17  Timezone               object 
 18  Airport_Code           object 
 19  Weather_Timestamp      object 
 20  Temperature(F)         float64
 21  Wind_Chill(F)          float64
 22  Humidity(%)       

In [19]:
us_accidents_df_new = us_accident_df_copy[["Start_Time", "City", "State", "Weather_Timestamp", "Temperature(F)", "Humidity(%)", "Pressure(in)", "Wind_Direction", "Wind_Speed(mph)", "Weather_Condition"]]

In [20]:
us_accidents_df_new.head()

,Start_Time,City,State,Weather_Timestamp,Temperature(F),Humidity(%),Pressure(in),Wind_Direction,Wind_Speed(mph),Weather_Condition
0,2016-02-08 05:46:00,Dayton,OH,2016-02-08 05:58:00,36.9,91.0,29.68,Calm,NaN,Light Rain
1,2016-02-08 06:07:59,Reynoldsburg,OH,2016-02-08 05:51:00,37.9,100.0,29.65,Calm,NaN,Light Rain
2,2016-02-08 06:49:27,Williamsburg,OH,2016-02-08 06:56:00,36.0,100.0,29.67,SW,3.5,Overcast
3,2016-02-08 07:23:34,Dayton,OH,2016-02-08 07:38:00,35.1,96.0,29.64,SW,4.6,Mostly Cloudy
4,2016-02-08 07:39:07,Dayton,OH,2016-02-08 07:53:00,36.0,89.0,29.65,SW,3.5,Mostly Cloudy


In [21]:
us_accidents_df_new.rename({"Wind_Speed(mph)": "Wind_Speed(m/s)", "Temperature(F)": "Temperature(K)", "Pressure(in)": "Pressure(hPa)"},axis=1, inplace =True)

In [22]:
us_accidents_df_new.head()

,Start_Time,City,State,Weather_Timestamp,Temperature(K),Humidity(%),Pressure(hPa),Wind_Direction,Wind_Speed(m/s),Weather_Condition
0,2016-02-08 05:46:00,Dayton,OH,2016-02-08 05:58:00,36.9,91.0,29.68,Calm,NaN,Light Rain
1,2016-02-08 06:07:59,Reynoldsburg,OH,2016-02-08 05:51:00,37.9,100.0,29.65,Calm,NaN,Light Rain
2,2016-02-08 06:49:27,Williamsburg,OH,2016-02-08 06:56:00,36.0,100.0,29.67,SW,3.5,Overcast
3,2016-02-08 07:23:34,Dayton,OH,2016-02-08 07:38:00,35.1,96.0,29.64,SW,4.6,Mostly Cloudy
4,2016-02-08 07:39:07,Dayton,OH,2016-02-08 07:53:00,36.0,89.0,29.65,SW,3.5,Mostly Cloudy


In [23]:
# Convert the 'datetime' column to datetime objects
us_accidents_df_new['Start_Time'] = pd.to_datetime(us_accidents_df_new['Start_Time'], format='ISO8601')

In [24]:
# Define the start date for filtering (January 1, 2019)
start_date = pd.to_datetime('2018-01-01')

# Filter the dataframe for dates greater than the start date
filtered_weather_df = us_accidents_df_new[us_accidents_df_new['Start_Time'] > start_date]

In [25]:
filtered_weather_df

,Start_Time,City,State,Weather_Timestamp,Temperature(K),Humidity(%),Pressure(hPa),Wind_Direction,Wind_Speed(m/s),Weather_Condition
512217,2022-09-08 05:49:30,Bartlett,IL,2022-09-08 05:52:00,58.0,90.0,29.24,CALM,0.0,Fair
512218,2022-09-08 02:02:05,Littlerock,CA,2022-09-08 01:53:00,86.0,28.0,27.35,W,6.0,Fair
512219,2022-09-08 05:14:12,Richmond,VA,2022-09-08 05:16:00,68.0,96.0,29.71,N,8.0,Mostly Cloudy
512220,2022-09-08 06:22:57,Alliance,OH,2022-09-08 06:51:00,62.0,86.0,28.71,NNE,6.0,Mostly Cloudy
512221,2022-09-08 06:36:20,Independence,OH,2022-09-08 06:53:00,63.0,87.0,29.37,SSE,3.0,Partly Cloudy
...,...,...,...,...,...,...,...,...,...,...
7728389,2019-08-23 18:03:25,Riverside,CA,2019-08-23 17:53:00,86.0,40.0,28.92,W,13.0,Fair
7728390,2019-08-23 19:11:30,San Diego,CA,2019-08-23 18:53:00,70.0,73.0,29.39,SW,6.0,Fair
7728391,2019-08-23 19:00:21,Orange,CA,2019-08-23 18:53:00,73.0,64.0,29.74,SSW,10.0,Partly Cloudy
7728392,2019-08-23 19:00:21,Culver City,CA,2019-08-23 18:51:00,71.0,81.0,29.62,SW,8.0,Fair


In [26]:
weather_df_2 = filtered_weather_df[filtered_weather_df['City'].isin(['San Diego', 'San Francisco', 'Los Angeles', 'San Antonio', 'Dallas', 'Houston', 'Chicago', 'Minneapolis', 'Indianapolis', 'Charlotte'])]

In [28]:
weather_df_2['Temperature(K)'] = (weather_df_2['Temperature(K)'] - 32) * 5/9 + 273.15
weather_df_2['Wind_Speed(m/s)'] = weather_df_2['Wind_Speed(m/s)'] * 0.44704
weather_df_2['Pressure(hPa)'] = weather_df_2['Pressure(hPa)'] * 33.86389

In [30]:
weather_df_2 = weather_df_2.reset_index(drop=True)

In [31]:
weather_df_2.head()

,Start_Time,City,State,Weather_Timestamp,Temperature(K),Humidity(%),Pressure(hPa),Wind_Direction,Wind_Speed(m/s),Weather_Condition
0,2022-09-08 06:17:04,Charlotte,NC,2022-09-08 05:54:00,418.850617,81.0,987.809671,NNE,1.798603,Cloudy
1,2022-09-08 06:14:56,Charlotte,NC,2022-09-08 06:34:00,418.541975,84.0,985.100560,NNE,1.598758,Mostly Cloudy
2,2022-09-08 06:29:31,Charlotte,NC,2022-09-08 06:34:00,418.541975,84.0,985.100560,NNE,1.598758,Mostly Cloudy
3,2022-09-08 06:39:53,Charlotte,NC,2022-09-08 06:34:00,418.541975,84.0,985.100560,NNE,1.598758,Mostly Cloudy
4,2022-09-08 05:04:41,Dallas,TX,2022-09-08 04:53:00,421.011111,82.0,996.275644,E,0.999224,Partly Cloudy


In [34]:
weather_df_2.drop({"Start_Time", "State", "Wind_Direction"}, axis=1, inplace=True)

In [35]:
weather_df_2.shape

(658133, 7)

# Concantenate 2 data sources

In [36]:
weather_df_1.head()

,Weather_Timestamp,City,Humidity(%),Pressure(hPa),Temperature(K),Weather_Condition,Wind_Speed(m/s)
0,2012-10-01 12:00:00,San Francisco,NaN,NaN,NaN,NaN,NaN
1,2012-10-01 13:00:00,San Francisco,88.0,1009.0,289.480000,light rain,2.0
2,2012-10-01 14:00:00,San Francisco,87.0,1009.0,289.474993,sky is clear,2.0
3,2012-10-01 15:00:00,San Francisco,86.0,1009.0,289.460618,sky is clear,2.0
4,2012-10-01 16:00:00,San Francisco,85.0,1009.0,289.446243,sky is clear,2.0


In [37]:
weather_df_2

,City,Weather_Timestamp,Temperature(K),Humidity(%),Pressure(hPa),Wind_Speed(m/s),Weather_Condition
0,Charlotte,2022-09-08 05:54:00,418.850617,81.0,987.809671,1.798603,Cloudy
1,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy
2,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy
3,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy
4,Dallas,2022-09-08 04:53:00,421.011111,82.0,996.275644,0.999224,Partly Cloudy
...,...,...,...,...,...,...,...
658128,Los Angeles,2019-08-23 13:52:00,422.554321,47.0,1004.064338,0.000000,Fair
658129,San Diego,2019-08-23 15:53:00,420.085185,62.0,995.259727,1.398913,Fair
658130,Los Angeles,2019-08-23 15:51:00,420.085185,66.0,1003.725700,1.598758,Fair
658131,San Diego,2019-08-23 16:51:00,420.393827,64.0,1009.143922,1.398913,Fair


## Concat 2 dataset

In [92]:
weather_df_final = pd.concat([weather_df_2, weather_df_1], axis=0, ignore_index=True)

In [93]:
weather_df_final

,City,Weather_Timestamp,Temperature(K),Humidity(%),Pressure(hPa),Wind_Speed(m/s),Weather_Condition
0,Charlotte,2022-09-08 05:54:00,418.850617,81.0,987.809671,1.798603,Cloudy
1,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy
2,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy
3,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy
4,Dallas,2022-09-08 04:53:00,421.011111,82.0,996.275644,0.999224,Partly Cloudy
...,...,...,...,...,...,...,...
1110658,Charlotte,2017-11-29 20:00:00,294.150000,21.0,1023.000000,1.000000,few clouds
1110659,Charlotte,2017-11-29 21:00:00,293.900000,21.0,1023.000000,1.000000,few clouds
1110660,Charlotte,2017-11-29 22:00:00,292.060000,24.0,1023.000000,1.000000,few clouds
1110661,Charlotte,2017-11-29 23:00:00,287.580000,38.0,1023.000000,0.000000,few clouds


## Filter Minority

In [99]:
import pandas as pd

# Assuming 'df' is your DataFrame and 'category_column' is the name of your categorical column.

# Step 1: Calculate the frequency of each category
category_counts = weather_df_final['Weather_Condition'].value_counts()

# Step 2: Set a threshold
# For example, you want to keep categories that appear at least 100 times
threshold = 1000

# Step 3: Filter categories based on threshold
# Get the names of categories that meet the threshold
categories_to_keep = category_counts[category_counts > threshold].index

# Filter the DataFrame to only include those categories
weather_df_filter_final = weather_df_final[weather_df_final['Weather_Condition'].isin(categories_to_keep)]

In [100]:
weather_df_filter_final.shape

(1093645, 7)

## Grouping

In [101]:
def map_to_group(description):
    description_lower = description.lower()
    if 'fair' in description_lower:
        return 'Fair'
    elif 'cloud' in description_lower or 'overcast' in description_lower:
        if 'overcast' in description_lower:
            return 'Overcast'
        else:
            return 'Cloudy'
    elif 'clear' in description_lower:
        return 'Clear'
    elif 'rain' in description_lower or 'storm' in description_lower:
        if 't-storm' in description_lower or 'thunder' in description_lower:
            return 'Thunderstorm'
        else:
            return 'Rain'
    elif 'drizzle' in description_lower:
        return 'Drizzle'
    elif 'fog' in description_lower:
        return 'Fog'
    elif 'mist' in description_lower:
        return 'Fog'
    elif 'thunder' in description_lower:
        return "Thunderstorm"
    elif 'snow' in description_lower:
        return 'Snow'
    elif 'haze' in description_lower:
        return 'Fog'
    else:
        print(description_lower)
        return 'Other'

In [102]:
weather_df_filter_final['Weather_Group'] = weather_df_filter_final['Weather_Condition'].apply(map_to_group)

In [104]:
weather_df_filter_final['Weather_Group'].value_counts()
weather_df_filter_final = weather_df_filter_final[weather_df_filter_final['Weather_Group'].isin(['Cloudy', 'Fair', 'Clear', 'Rain', 'Fog', 'Drizzle', 'Thunderstorm', 'Overcast'])]

In [105]:
weather_df_filter_final

,City,Weather_Timestamp,Temperature(K),Humidity(%),Pressure(hPa),Wind_Speed(m/s),Weather_Condition,Weather_Group
0,Charlotte,2022-09-08 05:54:00,418.850617,81.0,987.809671,1.798603,Cloudy,Cloudy
1,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy,Cloudy
2,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy,Cloudy
3,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy,Cloudy
4,Dallas,2022-09-08 04:53:00,421.011111,82.0,996.275644,0.999224,Partly Cloudy,Cloudy
...,...,...,...,...,...,...,...,...
1110658,Charlotte,2017-11-29 20:00:00,294.150000,21.0,1023.000000,1.000000,few clouds,Cloudy
1110659,Charlotte,2017-11-29 21:00:00,293.900000,21.0,1023.000000,1.000000,few clouds,Cloudy
1110660,Charlotte,2017-11-29 22:00:00,292.060000,24.0,1023.000000,1.000000,few clouds,Cloudy
1110661,Charlotte,2017-11-29 23:00:00,287.580000,38.0,1023.000000,0.000000,few clouds,Cloudy


## Final Cut

In [106]:
import pandas as pd

# Assuming you've already loaded your dataset into a DataFrame named df
# For example: df = pd.read_csv('your_data_file.csv')

# Convert the timestamp column to datetime format
weather_df_filter_final['Weather_Timestamp'] = pd.to_datetime(weather_df_filter_final['Weather_Timestamp'])

# Define the start and end dates for your filter
start_date = '2015-01-01'
end_date = '2022-12-31'

# Filter the DataFrame to include only the rows where the timestamp falls within the specified range
weather_df_final_filter = weather_df_filter_final[(weather_df_filter_final['Weather_Timestamp'] >= start_date) & (weather_df_filter_final['Weather_Timestamp'] <= end_date)]

# filtered_df now contains the data between 1st January 2015 and 31st December 2022


In [107]:
weather_df_final_filter

,City,Weather_Timestamp,Temperature(K),Humidity(%),Pressure(hPa),Wind_Speed(m/s),Weather_Condition,Weather_Group
0,Charlotte,2022-09-08 05:54:00,418.850617,81.0,987.809671,1.798603,Cloudy,Cloudy
1,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy,Cloudy
2,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy,Cloudy
3,Charlotte,2022-09-08 06:34:00,418.541975,84.0,985.100560,1.598758,Mostly Cloudy,Cloudy
4,Dallas,2022-09-08 04:53:00,421.011111,82.0,996.275644,0.999224,Partly Cloudy,Cloudy
...,...,...,...,...,...,...,...,...
1110658,Charlotte,2017-11-29 20:00:00,294.150000,21.0,1023.000000,1.000000,few clouds,Cloudy
1110659,Charlotte,2017-11-29 21:00:00,293.900000,21.0,1023.000000,1.000000,few clouds,Cloudy
1110660,Charlotte,2017-11-29 22:00:00,292.060000,24.0,1023.000000,1.000000,few clouds,Cloudy
1110661,Charlotte,2017-11-29 23:00:00,287.580000,38.0,1023.000000,0.000000,few clouds,Cloudy


## Upload to S3

In [108]:
bucket = 'big-data-rmit-asm3'
prefix = 'cleaned_data/'

# Create a S3 resource using boto3
s3_resource = boto3.Session().resource('s3')

def upload_s3_csv(filename, dataframe):
    """
    Upload a DataFrame to an S3 bucket as a CSV file

    :param filename: The filename to save as in the S3 bucket
    :param dataframe: The DataFrame to save
    """
    csv_buffer = io.StringIO()
    # Convert the DataFrame to CSV and save it to a buffer
    dataframe.to_csv(csv_buffer, header=True, index=False)

    # Full path for the file in the bucket
    full_file_path = os.path.join(prefix, filename)

    # Upload the CSV file to S3
    s3_resource.Bucket(bucket).Object(full_file_path).put(Body=csv_buffer.getvalue())
    print(f"File '{filename}' uploaded to '{full_file_path}' in bucket '{bucket}'.")

# Example usage
file_name = 'completed_dataset.csv'
# Assume df is your DataFrame that you want to upload
upload_s3_csv(file_name,weather_df_final_filter )


File 'completed_dataset.csv' uploaded to 'cleaned_data/completed_dataset.csv' in bucket 'big-data-rmit-asm3'.
